In [ ]:
#####Hierarchical Geographical Modeling of User Locationsfrom Social Media Posts#####
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import scipy
from scipy import stats
from scipy import special
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *

#np.random.seed(98537)

In [ ]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    if pattern==1:
        z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [ ]:
####データの生成####
##データの設定
geo = 2
levels = 2
k1 = 5
k2 = np.array([4, 4, 5, 5, 6])
topic = 15
hh = 3000
v = 1000   #語彙数
Lambda = np.random.gamma(12.5, 1/0.25, hh)
pt = np.random.poisson(Lambda, hh); pt[pt < 10] = 10
pt_vec = pt[:, np.newaxis]
N = np.sum(pt)
w = np.random.poisson(np.random.gamma(10.0, 1/0.5, N)); w[w < 5] = 5
f = np.sum(w)

In [ ]:
##IDとインデックスを作成
#IDを作成
u_id = np.repeat(np.arange(hh), pt)
d_id = np.repeat(np.arange(N), w)

#インデックスを作成
u_list = [i for i in range(hh)]
d_list = [i for i in range(N)]
d_vec = [i for i in range(N)]
for i in range(hh):
    u_list[i] = np.array(np.where(u_id==i)[0], dtype="int")
for i in range(N):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
    d_vec[i] = np.repeat(1, d_list[i].shape[0])

In [ ]:
####パラメータとデータの生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##ノード分布を生成
    #グローバルノード分布を生成
    G1 = np.random.dirichlet(np.repeat(3.0, k1), 1).reshape(-1)
    G2 = [np.random.dirichlet(np.repeat(2.5, k2[j]), 1).reshape(-1)  for j in range(k1)]
    GT1 = G1.copy(); GT2 = G2.copy()

    #ローカルノード分布を生成
    alpha1 = 2.0
    theta1 = np.random.dirichlet(alpha1*G1, hh)
    theta2 = [np.random.dirichlet(alpha1*G2[j], hh) for j in range(k1)]
    thetat1 = theta1.copy(); thetat2 = theta2.copy()

    
    ##モデルパラメータを生成
    #トピック分布を生成
    er = 0.01
    alpha2 = 2.5
    gamma_global = np.zeros((k1, topic))
    gamma = [j for j in range(k1)]
    for j in range(k1):
        gamma_global[j, ] = np.random.dirichlet(np.repeat(0.3, topic), 1)
        gamma[j] = np.random.dirichlet(alpha2*gamma_global[j, ]+er, k2[j])
    gammat_global = gamma_global.copy()
    gammat = gamma.copy()

    #location分布を生成
    min_point = -7.5; max_point = 7.5
    target_range = np.array([-2.75, 2.75])
    longitude = np.array([min_point, max_point]); latitude = np.array([min_point, max_point])
    mu_global = np.append(np.random.uniform(longitude[0], longitude[1], k1), 
                          np.random.uniform(latitude[0], latitude[1], k1)).reshape(k1, geo, order="F")
    mu = [i for i in range(k1)]
    Cov = [i for i in range(k1)]
    for i in range(k1):
        mu0 = np.zeros((k2[i], geo))
        Cov0 = np.zeros((geo, geo, k2[i]))
        for j in range(k2[i]):
            mu0[j, ] = mu_global[i, ] + np.random.uniform(target_range[0], target_range[1], geo)
            cov_temp = np.diag(np.random.uniform(0.05, 0.7, geo))
            value = np.random.uniform(-0.6, 0.6, 1) * np.prod(np.sqrt(np.diag(cov_temp)))
            cov_temp[0, 1] = value; cov_temp[1, 0] = value
            Cov0[:, :, j] = cov_temp
        mu[i] = mu0
        Cov[i] = Cov0
    mut_global = mu_global.copy()
    mut = mu.copy(); Covt = Cov.copy()

    #単語分布を生成
    m = 25
    beta = np.repeat(0.025, v)
    phi = np.random.dirichlet(beta, topic)
    index_v = np.array(range(v))[np.max(phi, axis=0) <= (topic*m)/f]
    for j in range(index_v.shape[0]):
        phi[np.argmax(np.random.multinomial(1, np.repeat(1/topic, topic), 1)), index_v[j]] = (topic*m)/f
    phi = phi / np.sum(phi, axis=1).reshape(topic, 1)
    phit = phi.copy()


    ##潜在変数を生成
    #多項分布から潜在ノードを生成
    R1 = np.array(rmnom(theta1[u_id, ], N, k1, np.arange(N), 1)[1].todense(), dtype="int8")
    r1 = np.dot(R1, np.arange(k1))
    R2 = np.zeros((N, np.max(k2)), dtype="int8")
    r2 = np.repeat(0, N)
    index_r1 = [j for j in range(k1)]
    n = np.repeat(0, k1)
    for j in range(k1):
        index_r1[j] = np.array(np.where(r1==j)[0], dtype="int")
        n[j] = index_r1[j].shape[0]
        R2[index_r1[j], :k2[j]] = np.array(rmnom(theta2[j][u_id[index_r1[j]], ] ,n[j], k2[j], np.arange(n[j]), 1)[1].todense(), dtype="int8")
        r2[index_r1[j]] = np.dot(R2[index_r1[j], :k2[j]], np.arange(k2[j]))
    r = np.array(np.hstack((r1[:, np.newaxis], r2[:, np.newaxis])), dtype="int16")

    #トピック分布を生成
    r_vec = r[d_id, ]
    index_r2 = [j for j in range(k1)]
    gamma_dt = np.zeros((f, topic))
    for j in range(k1):
        index_r2[j] = np.array(np.where(r_vec[:, 0]==j)[0], dtype="int")
        gamma_dt[index_r2[j], ] = gamma[j][r_vec[index_r2[j], 1], ]

    #多項分布からトピックを生成
    out = rmnom(gamma_dt, f, topic, np.arange(f), 1)
    Z = np.array(out[1].todense(), dtype="int8")
    z_vec = np.array(out[0], dtype="int16")


    ##データを生成
    #多変量正規分布からlocationを生成
    no = np.arange(N)
    y = np.zeros((N, geo))
    for i in range(k1):
        index1 = index_r1[i]
        for j in range(k2[i]):
            index2 = np.where(r[index1, 1]==j)[0]
            y[index1[index2], ] = np.random.multivariate_normal(mu[i][j, ], Cov[i][:, :, j], index2.shape[0])

    #多項分布から単語を生成
    wd = np.array(np.repeat(0, f), dtype="int16")
    for i in range(N):
        index = d_list[i]
        wd[index] = np.array(rmnom(phi[z_vec[index], ], w[i], v, np.arange(w[i]), 0), dtype="int16")

    #単語がすべて生成されればbreak
    if np.unique(wd).shape[0]==v:
        break

In [ ]:
#locatio分布を可視化
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
fig = plt.figure(figsize=(12.0, 6.0))
for j in range(k1):
    index = index_r1[j]
    ax = fig.add_subplot(2, 3, j+1)
    ax.scatter(y[index, 0], y[index, 1], s=1, c=colorlist[j], alpha=0.25, linewidths="1", edgecolors=colorlist[j])
    plt.title("location point", fontsize=12.5)
    plt.xlabel("longitude")
    plt.ylabel("latitude")
ax = fig.add_subplot(2, 3, k1+1)
ax.scatter(y[:, 0], y[:, 1], s=1, c=colorlist[k1+1], alpha=0.25, linewidths="1", edgecolors=colorlist[k1+1])
plt.title("location point", fontsize=12.5)
plt.xlabel("longitude")
plt.ylabel("latitude")
fig.tight_layout()
plt.show()

In [ ]:
#単語のインデックスを作成
wd_list = [i for i in range(v)]
wd_vec = [i for i in range(v)]
word_freq = np.repeat(0, v)
for i in range(v):
    wd_list[i] = np.array(np.where(wd==i)[0], dtype="int")
    wd_vec[i] = np.repeat(1, wd_list[i].shape[0])
    word_freq[i] = wd_list[i].shape[0]

In [ ]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##多変量正規分布の密度関数
def dmv(x, mu, Cov, k):
    er = x - mu
    Cov_inv = np.linalg.inv(Cov) 
    LLo = 1 / (np.sqrt(pow((2 * np.pi), k) * np.linalg.det(Cov))) * np.exp(-1/2 * np.dot(np.dot(er, Cov_inv) *er, np.ones(k)))
    return(LLo)

##アルゴリズムの設定
k = 10
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10
target_iter = 50

In [ ]:
##事前分布の設定
#nCRFの集中度パラメータ
alpha = 2.5
beta = 1.0

#モデルパラメータの事前分布
Deltabar = np.repeat(0.0, geo)
ADelta = 0.01
nu = geo + 1
V = nu * np.diag(np.ones(geo))
alpha1 = 0.25
alpha2 = 0.1

In [ ]:
##パラメータの真値
#ノード構造の真値
levels = 2
k1 = 5
k2 = np.array([4, 4, 5, 5, 6])

#潜在変数の真値
Ri1 = np.array(R1.copy(), dtype="int")
r1 = np.dot(Ri1, np.arange(k1))
Ri2 = np.array(R2.copy(), dtype="int")
r2 = np.dot(Ri2, np.arange(np.max(k2)))
r = np.hstack((r1[:, np.newaxis], r2[:, np.newaxis]))
Zi = Z.copy()
z = np.dot(Zi, np.arange(topic))

#モデルパラメータの真値
G1 = GT1.copy(); G2 = GT2.copy()
theta1 = thetat1.copy(); theta2 = thetat2.copy()
gamma = gammat.copy()
mu = mut.copy(); Cov = Covt.copy()
phi = phit.copy()

#location分布の真値
mu_global = np.zeros((k1, geo))
Cov_global = np.zeros((geo, geo, k1))
for j in range(k1):
    index = index_r1[j]
    mu_global[j, ] = np.mean(y[index, ], axis=0)
    er = y[index, ] - mu_global[j, ]
    Cov_global[:, :, j] = np.dot(er.T, er) / index.shape[0]

In [ ]:
##初期値の設定
#ノード構造の初期値
levels = 2
k1 = 5
k2 = np.array([4, 4, 5, 5, 6])

#モデルパラメータの初期値
alpha1 = 2.0; alpha2 = 2.5
G1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1)
G2 = [np.random.dirichlet(np.repeat(5.0, k2[j]), 1).reshape(-1)  for j in range(k1)]
GT1 = G1.copy(); GT2 = G2.copy()
theta1 = np.random.dirichlet(alpha1*G1, hh)
theta2 = [np.random.dirichlet(alpha1*G2[j], hh) for j in range(k1)]
gamma_global = np.zeros((k1, topic))
gamma = [j for j in range(k1)]
for j in range(k1):
    gamma_global[j, ] = np.random.dirichlet(np.repeat(5.0, topic), 1)
    gamma[j] = np.random.dirichlet(alpha2*gamma_global[j, ]+0.01, k2[j])
mu = [i for i in range(k1)]
Cov = [i for i in range(k1)]
for i in range(k1):
    mu0 = np.zeros((k2[i], geo))
    Cov0 = np.zeros((geo, geo, k2[i]))
    for j in range(k2[i]):
        mu0[j, ] = np.random.uniform(-5.0, 5.0, geo)
        Cov0[:, :, j] = np.diag(np.repeat(0.5, geo))
    mu[i] = mu0
    Cov[i] = Cov0
phi = np.random.dirichlet(np.repeat(5.0, v), topic)

#潜在ノードの初期値
Ri1 = np.array(rmnom(theta1[u_id, ], N, k1, np.arange(N), 1)[1].todense(), dtype="int8")
r1 = np.dot(Ri1, np.arange(k1))
Ri2 = np.zeros((N, np.max(k2)), dtype="int8")
r2 = np.repeat(0, N)
index_r1 = [j for j in range(k1)]
for j in range(k1):
    index = np.array(np.where(Ri1[:, j]==1)[0], dtype="int")
    n = index.shape[0]
    Ri2[index, :k2[j]] = np.array(rmnom(theta2[j][u_id[index], ], n, k2[j], np.arange(n), 1)[1].todense(), dtype="int8")
    r2[index] = np.dot(R2[index, :k2[j]], np.arange(k2[j]))

#トピックの初期値
out = rmnom(np.full((f, topic), 1/topic), f, topic, np.arange(f), 1)
Zi = np.array(out[1].todense(), dtype="int8")
z_vec = np.array(out[0], dtype="int16")

In [ ]:
##nCRFのグローバルノードとローカルノードの頻度を定義
#パラメータの格納用配列
M1 = np.sum(Ri1, axis=0)
M2 = [i for i in range(k1)]
m1 = np.zeros((hh, k1), dtype="int")
m2 = [np.zeros((hh, k2[i]), dtype="int") for i in range(k1)]

#グローバルノードの頻度
freq = np.dot(Ri2.T, Ri1).T
for j in range(k1):
    M2[j] = freq[j, ][:k2[j]]
    
#ローカルノードの頻度
for i in range(hh):
    index = u_list[i]
    m1[i, ] = np.sum(Ri1[index, ], axis=0)
    freq = np.dot(Ri2[index, ].T, Ri1[index, ]).T
    for j in range(k1):
        m2[j][i, ] = freq[j, ][:k2[j]]
        
#階層的事前分布を定義
Prior1 = m1/(pt_vec+alpha) + alpha/(pt_vec+alpha)*M1/(N+beta)
Prior2 = [j for j in range(k1)]
for j in range(k1):
    freq = np.sum(m2[j], axis=1)[:, np.newaxis]
    Prior2[j] = m2[j]/(freq+alpha) + alpha/(freq+alpha)*M2[j]/(np.sum(M2[j])+beta)
    
#文書ごとのトピック頻度を数える
z_freq = np.zeros((N, topic), dtype="int")
for i in range(N):
    z_freq[i, ] = np.dot(Zi[d_list[i], ].T, d_vec[i])

In [ ]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
MU = [j for j in range(int(R/keep))]
COV = [j for j in range(int(R/keep))]
GAMMA = [j for j in range(int(R/keep))]
PHI = np.zeros((topic, v, int(R/keep)))

#潜在変数の格納用配列
SEG1 = np.zeros((N, int(R/keep)), dtype="int8")
SEG2 = np.zeros((N, int(R/keep)), dtype="int")
TOPIC = np.zeros((f, topic), dtype="int")

In [ ]:
##学習データの真値の対数尤度
#locationの対数尤度
LL01 = 0
for i in range(k1):
    index = np.array(np.where(R1[:, i])[0], dtype="int")
    Lho_dmv = np.zeros((index.shape[0], k2[i]))
    for j in range(k2[i]):
        Lho_dmv[:, j] = dmv(y[index, ], mut[i][j, ], Covt[i][:, :, j], geo)
    LL01 += np.sum(np.log(np.sum(thetat2[i][u_id[index], ] * Lho_dmv, axis=1)))

#トピックモデルの対数尤度
R_vec01 = R1[d_id, ]; R_vec02 = R2[d_id, ]
r_vec02 = np.dot(R_vec02, np.arange(np.max(k2)))
Lho_topic = np.zeros((f, topic))
for j in range(k1):
    index = np.array(np.where(R_vec01[:, j])[0], dtype="int")
    Lho_topic[index, ] = gammat[j][r_vec02[index], ] * (phit.T)[wd[index], ]
LL02 = np.sum(np.log(np.sum(Lho_topic, axis=1)))
LL0 = LL01 + LL02

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##ノードをサンプリング
    #同時分布の尤度の格納用配列
    Lho_joint1 = np.zeros((N, k1))
    Lho_joint2 = [i for i in range(k1)]
    Lho_dmv = [i for i in range(k1)]

    #ノードごとの尤度を定義
    for i in range(k1):
        Lho1 = np.zeros((N, k2[i]))
        for j in range(k2[i]):
            #level2のlocation分布の尤度
            Lho1[:, j] = dmv(y, mu[i][j, ], Cov[i][:, :, j], geo)

        #level2のトピック分布の尤度
        Lho2 = np.exp(np.dot(z_freq, np.log(gamma[i]).T))

        #level1のlocation分布とトピック分布の期待尤度
        Lho_dmv[i] = Prior2[i][u_id, ] * Lho1
        Lho_joint2[i] = Prior2[i][u_id, ] * Lho1 * Lho2
        Lho_joint1[:, i] = Prior1[u_id, i] * np.sum(Lho_joint2[i], axis=1)

    #多項分布からlevel1のノードをサンプリング
    Prob1 = Lho_joint1 / np.sum(Lho_joint1, axis=1)[:, np.newaxis]
    out = rmnom(Prob1, N, k1, no, 1)
    Ri1 = np.array(out[1].todense(), dtype="int")
    r1 = np.dot(Ri1, np.arange(k1))

    #多項分布からlevel2のノードをサンプリング
    index_r1 = [j for j in range(k1)]
    Ri2 = np.zeros((N, np.max(k2)), dtype="int")
    for j in range(k1):
        index_r1[j] = np.array(np.where(Ri1[:, j]==1)[0], dtype="int")
        Lho_target = Lho_joint2[j][index_r1[j], ]
        Prob2 = Lho_target / np.sum(Lho_target, axis=1)[:, np.newaxis]
        Ri2[index_r1[j], :k2[j]] = np.array(rmnom(Prob2, index_r1[j].shape[0], k2[j], 
                                                  np.arange(index_r1[j].shape[0]), 1)[1].todense(), dtype="int")
    r2 = np.dot(Ri2, np.arange(np.max(k2)))


    ##nCRFのグローバルノードとローカルノードの頻度を定義
    #パラメータの格納用配列
    M1 = np.sum(Ri1, axis=0)
    M2 = [i for i in range(k1)]
    m1 = np.zeros((hh, k1), dtype="int")
    m2 = [np.zeros((hh, k2[i]), dtype="int") for i in range(k1)]

    #グローバルノードの頻度
    freq = np.dot(Ri2.T, Ri1).T
    for j in range(k1):
        M2[j] = freq[j, ][:k2[j]]

    #ローカルノードの頻度
    for i in range(hh):
        index = u_list[i]
        m1[i, ] = np.sum(Ri1[index, ], axis=0)
        freq = np.dot(Ri2[index, ].T, Ri1[index, ]).T
        for j in range(k1):
            m2[j][i, ] = freq[j, ][:k2[j]]

    #階層的事前分布を定義
    Prior1 = m1/(pt_vec+alpha) + alpha/(pt_vec+alpha)*M1/(N+beta)
    Prior2 = [j for j in range(k1)]
    for j in range(k1):
        freq = np.sum(m2[j], axis=1)[:, np.newaxis]
        Prior2[j] = m2[j]/(freq+alpha) + alpha/(freq+alpha)*M2[j]/(np.sum(M2[j])+beta)


    ##ノードごとにlocation分布のパラメータをサンプリング
    #多変量正規分布から期待値をサンプリング
    for i in range(k1):
        index = index_r1[i]
        for j in range(k2[i]):
            y_target = y[index, ] * Ri2[index, j][:, np.newaxis]
            mu_par = np.sum(y_target, axis=0) / (M2[i][j] + ADelta)
            Sigma = np.kron(Cov[i][:, :, j], 1/M2[i][j])
            mu[i][j, ] = np.random.multivariate_normal(mu_par, Sigma, 1)

            #逆ウィシャート分布から分散共分散行列をサンプリング
            er = y_target - mu[i][j, ]*Ri2[index, j][:, np.newaxis]
            IW_R = np.dot(er.T, er) + V
            Sn = M2[i][j, ] + nu
            Cov[i][:, :, j] = scipy.stats.invwishart.rvs(Sn, IW_R, 1)


    ##ノードごとにトピックをサンプリング
    #ノード割当を単語idに割り当てる
    R_vec1 = Ri1[d_id, ]; R_vec2 = Ri2[d_id, ]
    r_vec2 = np.dot(R_vec2, np.arange(np.max(k2)))

    #トピックの割当確率を定義
    Lho_topic = np.zeros((f, topic))
    index_r2 = [j for j in range(k1)]
    for j in range(k1):
        index_r2[j] = np.array(np.where(R_vec1[:, j])[0], dtype="int")
        Lho_topic[index_r2[j], ] = gamma[j][r_vec2[index_r2[j]], ] * (phi.T)[wd[index_r2[j]], ]
    Prob = Lho_topic / np.sum(Lho_topic, axis=1)[:, np.newaxis]

    #多項分布からトピックをサンプリング
    out = rmnom(Prob, f, topic, np.arange(f), 1)
    Zi = np.array(out[1].todense(), dtype="int")
    z = np.array(out[0], dtype="int")

    #文書ごとのトピック頻度を数える
    z_freq = np.zeros((N, topic), dtype="int")
    for i in range(N):
        z_freq[i, ] = np.dot(Zi[d_list[i], ].T, d_vec[i])


    ##トピックモデルのパラメータをサンプリング
    #トピック分布をサンプリング
    for i in range(k1):
        wsum = np.dot(Zi.T, R_vec2[:, :k2[i]] * R_vec1[:, i][:, np.newaxis]).T + alpha1
        for j in range(k2[i]):
            gamma[i][j, ] = np.random.dirichlet(wsum[j, ], 1).reshape(-1)

    #単語分布をサンプリング
    vsum = np.zeros((topic, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi[wd_list[j], ].T, wd_vec[j])
    for j in range(topic):
        phi[j, ] = np.random.dirichlet(vsum[j, ] + beta, 1).reshape(-1)


    ##サンプリング結果の格納と表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = int(rp/keep)
        MU[mkeep] = mu        
        COV[mkeep] = Cov
        GAMMA[mkeep] = gamma
        PHI[:, :, mkeep] = phi
        SEG1[:, mkeep] = r1
        SEG2[:, mkeep] = r2
        
    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        TOPIC = TOPIC + Zi
    
    if rp%disp==0:
        #学習データの対数尤度
        LL1 = 0.0
        for j in range(k1):
            LL1 += np.sum(np.log(np.sum(Lho_dmv[j][index_r1[j], ], axis=1)))
        LL2 = np.sum(np.log(np.sum(Lho_topic, axis=1)))
        LL = LL1 + LL2

        #サンプリング結果の表示
        print(rp)
        print(np.round([LL1, LL2, LL, LL01, LL02, LL0], 1))